In [ ]:
"""
Author: Ryleigh J. Bruce
Date: June 28, 2024

Purpose: To extract data regarding close encounters between different animal species and store the data in an Excel sheet, which can then be used to generate a plot to visualize the key data.


Note: The author generated this text in part with GPT-4,
OpenAI’s large-scale language-generation model. Upon generating
draft code, the authors reviewed, edited, and revised the code
to their own liking and takes ultimate responsibility for
the content of this code.

"""

## Module: Mount the Notebook to Google Drive

Here we import the drive module that allows us to link the Colab environment with our google drive, where the desired data set is stored. This allows us to access any files located within Google Drive and interact with them directly.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Organizing Species Observations into Sequences

Please note that the following sequencing script was originally run in VS Code due to the high runtime required to run the script in Google Colab.

## Module: Importing the Necessary Libaries

In order to run the script the `Pandas` library and `timedelta` class from the `datetime` module must be imported. These will be critical for data analysis and manipulation.

The `timedelta` class is used to represent a duration, such as the difference between two times.

In [ ]:
import pandas as pd
from datetime import timedelta

## Module: Reading the Excel File

Here the file paths for the source data file and the Excel file that will be generated are defined. If the name of the new file being created should be changed, simply update `sequenced_sightingas_data.xlsx` to the desired file name.

Next the Pandas `pd` library is used to read the Excel file and transform it into a DataFrame, and then assigns it into the `df` variable.

In [ ]:
# Define file paths
input_file = r"C:\Users\rjbru\OneDrive\Desktop\2024_URA\Understanding Animals\Data Sets\combined_csv_animal_flag_justanimals_location_flat (1).xlsx"
output_file = r"C:\Users\rjbru\OneDrive\Desktop\2024_URA\Understanding Animals\Data Sets\sequenced_sightings_data.xlsx"

# Load the existing Excel file
df = pd.read_excel(input_file)

## Module: Manipulating the DataFrame

The `Date` and `Time` columns in the newly created DataFrame are combined into a single `datetime` column, which is required for the time analysis to be performed later in the script.

In [ ]:
# Combine 'Date' and 'Time' into a single 'datetime' column
df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))

Next the DataFrame is organized by the `SpeciesList`, `locationID`, `cameraNum`, and `datetime` columns in the order specified. This will make further data analysis to determine close encounters more efficient.

In [ ]:
# Sort data by SpeciesList, locationID, cameraNum, and datetime
df.sort_values(by=['SpeciesList', 'locationID', 'cameraNum', 'datetime'], inplace=True)

## Module: Grouping the Data into Sequences

Next the `group_sequences()` function must be defined.

It begins by initializing an empty `sequences` list to store the sequence data. Then `sequence_start_index` is implemented to keep track of the starting index of the current sequence, so that the number of images within a given sequence may be recorded.

The function then iterates through rows within the DataFrame and compares each row to the previous row to check if the `SpeciesList`, `locationID`, and `cameraNum` values are the same and if the difference between the `datetime` fields is equal to or less than one minute. If any of the values are different or if the time difference is more than a minute it indicates the end of a sequence.


After each sequence has been completed it is appended to the `sequences` list.


After the final sequence has been recorded, the script returns the completed list of sequences.

In [ ]:
def group_sequences(df):
    sequences = []
    sequence_start_index = 0

    for i in range(1, len(df)):
        current_row = df.iloc[i]
        previous_row = df.iloc[i - 1]

        # Check if the current row is a continuation of the sequence
        same_species = current_row['SpeciesList'] == previous_row['SpeciesList']
        same_location = current_row['locationID'] == previous_row['locationID']
        same_camera = current_row['cameraNum'] == previous_row['cameraNum']
        within_one_minute = (current_row['datetime'] - previous_row['datetime']) <= timedelta(minutes=1)

        if not (same_species and same_location and same_camera and within_one_minute):
            # End the current sequence and start a new one
            sequences.append(df.iloc[sequence_start_index:i].copy())
            sequence_start_index = i

    # Add the last sequence
    sequences.append(df.iloc[sequence_start_index:].copy())

    return sequences

The newly defined `group_sequences()` function is then used to sort the DataFrame and assign the modified DataFrame to the `sequences` variable.

In [ ]:
# Group the data into sequences
sequences = group_sequences(df)

An empty `grouped_data` list is then initialized to store the grouped data.

In [ ]:
# Create a new DataFrame to store the grouped data
grouped_data = []

This ‘for’ block  iterates over each sequence and copies the first row while adding the `sequence_count` row to indicate the number of images within a sequence. This data is then appended to the `grouped_data` variable.

In [ ]:
for sequence in sequences:
    first_entry = sequence.iloc[0].copy()  # Copy to avoid SettingWithCopyWarning
    first_entry['sequence_count'] = len(sequence)  # The number of images in the sequence
    grouped_data.append(first_entry)

Next the data stored in the `grouped_data` is converted to a DataFrame.

In [ ]:
grouped_df = pd.DataFrame(grouped_data)

The format of the `Date` column in the DataFrame is specified to be 'year-month-day’ to match the other data columns.

In [ ]:
# Ensure the Date column is in "Year-Month-Day" format
grouped_df['Date'] = pd.to_datetime(grouped_df['Date']).dt.strftime('%Y-%m-%d')

## Module: Creating a New Excel File

The `grouped_df` is then saved to a new Excel file using the pandas `to_excel` method.

In [ ]:
# Save the grouped data to a new Excel file
grouped_df.to_excel(output_file, index=False)

A print statement specifies the file path of the newly saved Excel file. This file will be used as the data file for the following scripts.

In [ ]:
print(f'Grouped data saved to {output_file}')

# Documenting Interspecies and Intraspecies Close Encounters

The script outlined in this section analyzes the sequenced data and creates a new Excel file containing close encounters between animals of the same species, as well as animals from different species.

## Module: Importing the Necessary Libraries

The script requires the `os`, `pandas`, and `datetime` modules as well as the `timedelta` class. These aid in navigating and creating file directories, data manipulation, and analyzing date and time information.

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

The source file path and output directory paths are assigned to the `file_path` and `output_dir` variables. `output_file_path` uses the `os.path.join()` function to append the desired file name to the end of the `output_path`.

In [ ]:
# File paths
file_path = "/content/drive/MyDrive/shared-data/Notebook datafiles/Close-encounters/sequenced_sightings_data.xlsx"  # Change this to your file path
output_dir = "/content/drive/MyDrive/shared-data/Notebook datafiles/Close-encounters"  # New directory path
output_file_path = os.path.join(output_dir, 'combined_species_encounters.xlsx')

This small ‘if’ block uses the `os` library to check if the output directory already exists, and creates one if it does not (along with any necessary parent directories).

In [ ]:
# Create the directory if it doesn’t exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

The source data file is then read and assigned to the `df` variable using the pandas `pd.read_excel()` function.

In [ ]:
# Load the Excel file
df = pd.read_excel(file_path)

## Module: Manipulating the DataFrame

The script then converts the `Date` and `Time` columns to the appropriate datetime data types for time data analysis.

In [ ]:
# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

These columns are then combined into a single `DateTime` column for easier data analysis and manipulation.

In [ ]:
# Combine Date and Time back into a single datetime column for easier manipulation
df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))

Next the DataFrame is sorted by the `locationID` and `DateTime` columns in ascending order.

In [ ]:
# Sort the dataframe by locationID, DateTime
df.sort_values(by=['locationID', 'DateTime'], inplace=True)

An empty `results` list is initialized to store close encounter data as it is collected.

In [ ]:
# Initialize list to store results
results = []

A unique counter is also initialized so that a unique encounter ID can be assigned to each close encounter that will be logged in the final Excel file.

In [ ]:
# Initialize unique counter for UniqueEncounterID
unique_counter = 1

## Module: Extracting Close Encounter Data

In this instance a close encounter is defined as animals being observed at the same location on the same day within ten minutes of each other (excluding instances that occur within a single sequence).

The script begins by grouping the DataFrame by the `locationID` value, creating a subset for each unique `locationID`.

The script then iterates over each subset and compares pairs of rows, determining the species and the time difference between sightings. This script records two types of encounters, encounters between the same species and encounters between different species. If a close encounter is discovered then its details are recorded along with a `unique_counter_id`.

In [ ]:
# Process each locationID
for location_id, group in df.groupby(['locationID']):
    group = group.reset_index(drop=True)
    for i in range(len(group) - 1):
        species_i = group.loc[i, 'SpeciesList']
        time_i = group.loc[i, 'DateTime']

        for j in range(i + 1, len(group)):
            species_j = group.loc[j, 'SpeciesList']
            time_j = group.loc[j, 'DateTime']

            # Calculate the time difference
            time_diff = abs((time_j - time_i).total_seconds()) / 60  # in minutes

            # Encounters: more than 2 to less than 10 minutes apart for same species and within 10 minutes for different species
            if (2 < time_diff < 10 and species_i == species_j) or (time_diff <= 10 and species_i != species_j):
                unique_encounter_id = f"{location_id}_{time_i.strftime('%Y%m%d')}_{unique_counter}"
                unique_counter += 1
                results.append({
                    'LocationID': location_id,
                    'Date': time_i.date(),
                    'Species1': species_i,
                    'Species2': species_j,
                    'Time1': time_i.time(),
                    'Time2': time_j.time(),
                    'UniqueEncounterID': unique_encounter_id
                })

            # Efficiently terminate the loop for intraspecies if greater than 10 minutes
            if time_diff > 10:
                break

## Module: Saving to a New Excel File

Here the `results` list containing the close encounters data is converted into a two-dimensional DataFrame.

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

The DataFrame is then converted to an Excel file using the `to_excel()` function from the Pandas library.

In [ ]:
# Write the results to an Excel file
results_df.to_excel(output_file_path, index=False)


This print statement indicates the path to the newly saved Excel file.

In [ ]:
print(f"Results written to {output_file_path}")

# Documenting Interspecies Close Encounters

The script outlined in this section analyzes the sequenced data and creates a new Excel file containing close encounters between animals from different species.

Since this script is performing relatively the same function as the previous close encounters script, the majority of the code will remain the same.

The same libraries are downloaded and the information is converted to a DataFrame that is sorted by `locationID` and `DateTime`.

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

# File paths
file_path = "/content/drive/MyDrive/shared-data/Notebook datafiles/Close-encounters/sequenced_sightings_data.xlsx"  # Change this to your file path
output_dir = "/content/drive/MyDrive/shared-data/Notebook datafiles/Close-encounters"  # New directory path
output_file_path = os.path.join(output_dir, 'combined_species_encounters.xlsx')

# Create the directory if it doesn’t exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the Excel file
df = pd.read_excel(file_path)

# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Combine Date and Time back into a single datetime column for easier manipulation
df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))

# Sort the dataframe by locationID, DateTime
df.sort_values(by=['locationID', 'DateTime'], inplace=True)

# Initialize list to store results
results = []

# Initialize unique counter for UniqueEncounterID
unique_counter = 1

This script, however, only records close encounters between animals of different species. In this instance a close encounter is defined as animals of different species being observed at the same location on the same day within ten minutes of each other.


This code block operates very similarly to the one in the previous script, grouping the DataFrame by `locationID` and iterating over each subset and appending close encounters to the `results` list when found.

In [ ]:
# Process each locationID
for location_id, group in df.groupby(['locationID']):
    group = group.reset_index(drop=True)
    for i in range(len(group) - 1):
        species_i = group.loc[i, 'SpeciesList']
        time_i = group.loc[i, 'DateTime']

        for j in range(i + 1, len(group)):
            species_j = group.loc[j, 'SpeciesList']
            time_j = group.loc[j, 'DateTime']

            # Calculate the time difference
            time_diff = abs((time_j - time_i).total_seconds()) / 60  # in minutes

            # Record encounters of different species within 10 minutes apart
            if time_diff <= 10 and species_i != species_j:
                unique_encounter_id = f"{location_id}_{time_i.strftime('%Y%m%d')}_{unique_counter}"
                unique_counter += 1
                results.append({
                    'LocationID': location_id,
                    'Date': time_i.date(),
                    'Species1': species_i,
                    'Species2': species_j,
                    'Time1': time_i.time(),
                    'Time2': time_j.time(),
                    'UniqueEncounterID': unique_encounter_id
                })

            # Efficiently terminate the loop if time difference is greater than 10 minutes
            if time_diff > 10:
                break

The `results` list is then converted to a DataFrame and saved as an Excel file.

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Write the results to an Excel file
results_df.to_excel(output_file_path, index=False)

print(f"Results written to {output_file_path}")

# Plotting Close Encounters Data

This script extracts data from the newly generated Excel file and visualizes it in the form of a network graph.

## Module: Importing the Necessary Libraries

Importing the following libraries will enable the script to read and write files, create and manipulate network graphs, generate plots and customizable visual elements, manipulate image data, and handle array data structures.

In [ ]:
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import PathPatch, FancyBboxPatch, BoxStyle
from matplotlib.lines import Line2D
from matplotlib.path import Path
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from PIL import Image
import numpy as np

Here the paths to the previously generated close encounters Excel file and the directory containing silhouette image files (if applicable) are defined. The data in the Excel file is loaded into a DataFrame for easier analysis and manipulation later in the script.

In [ ]:
# File paths
results_file_path = "/content/drive/MyDrive/shared-data/Notebook datafiles/Close-encounters/combined_species_encounters.xlsx"
silhouettes_dir = "/content/drive/MyDrive/shared-data/Notebook datafiles/Close-encounters/silhouettes"  # Directory containing species silhouette images

# Load the results from the previous Excel file
df = pd.read_excel(results_file_path)

These print statements print the first five rows of the DataFrame to ensure it has been loaded correctly.

In [ ]:
# Verify the data loaded correctly
print("Data preview:")
print(df.head())

## Module: Preparing the Graph

Here an empty graph is initialized and assigned to the variable `G`. Within this graph object the network graph will be constructed by adding nodes (species) and edges (encounters between species).

In [ ]:
# Create the network graph
G = nx.Graph()

This code block establishes unique colors to represent each locationID within the DataFrame.

The function `df[‘LocationID’].unique()` extracts an array of unique values from the `locationID` column and assigns those values to the `unique_location_ids` variable. The `get_cmap` function is then used to retrieve the `tab20c` color map. `len(unique_location_ids)` specifies the number of colors required for the graph. The color map is then assigned to the `colors` variable, which is now a color map object that can be used to get a unique color for each `locationID`.

In [ ]:
# Define a list of colors for different LocationIDs
unique_location_ids = df['LocationID'].unique()
colors = plt.get_cmap('tab20c', len(unique_location_ids))

This line of code creates the `dictionary location_color_map` where the keys are locationIDs and the values are colors, allowing each unique location to be visually differentiated in the network graph. These locationID color pairs are created using a dictionary comprehension.

In [ ]:
# Prepare color mapping for each unique LocationID
location_color_map = {loc_id: colors(i) for i, loc_id in enumerate(unique_location_ids)}

Here an empty `edge_counters` dictionary is initialized to track how many times each edge (close encounter) is added to the graph. This ensures that each line is manipulated to have unique curvature and prevent potential obstruction of edges.

In [ ]:
# Track how many times each edge is added to ensure unique curvature and volume depiction for each encounter
edge_counters = {}

This code block iterates over each row of the DataFrame and extracts the species, locationID, and associated location color. `edge = tuple(sorted((species1, species2)))` creates an edge between each species involved in the close encounter. `G.add_edge()` adds an edge between species and species2 to the graph object G.

In [ ]:
# Build the graph and assign colors
for index, row in df.iterrows():
    species1, species2 = row['Species1'], row['Species2']
    location_id = row['LocationID']
    color = location_color_map[location_id]

    edge = tuple(sorted((species1, species2)))  # Ensure consistent edge representation
    if edge not in edge_counters:
        edge_counters[edge] = 0

    edge_counters[edge] += 1
    G.add_edge(species1, species2, color=color, weight=edge_counters[edge])

Here the `networkx` library is used to arrange the nodes within graph `G` in a circular format with a radius of `2` units. This helps create a clear network graph.

In [ ]:
# Node positions in a circular layout with increased radius
pos = nx.circular_layout(G, scale=2)

Here a 20x20” plotting space is created using the `matplotlib` library. This ensures that there is sufficient space for the detailed network graph to be plotted in a legible manner.

In [ ]:
# Draw the network graph
fig, ax = plt.subplots(figsize=(20, 20))  # Increased figure size

## Module: Loading Silhouette Files

In order to represent the species nodes with silhouettes, the silhouette image files must be obtained from the specified directory and normalized so that they are relatively consistent across nodes.

The `get_image()` function is defined to load an image from the directory and then convert it to `RBGA` mode. This allows the script to process the image so that all white pixels are fully transparent, if not already. The image is then resized to the desired height while maintaining the same aspect ratio to minimize distortion, and the function then returns an `OffsetImage` object to be used in plotting the network graph.

In [ ]:
# Helper function to import and convert images
def get_image(path, target_height):
    try:
        img = Image.open(path)
        img = img.convert("RGBA")

        datas = img.getdata()
        new_data = []
        for item in datas:
            if item[0] > 200 and item[1] > 200 and item[2] > 200:
                new_data.append((255, 255, 255, 0))
            else:
                new_data.append(item)
        img.putdata(new_data)

        width, height = img.size
        new_width = int((target_height / height) * width)
        img = img.resize((new_width, target_height), Image.LANCZOS)

        return OffsetImage(img)
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

This ‘for’ block iterates over each node in the network graph and adds the species silhouette, or displays the species name. `for node, (x, y) in pos.items():` starts the loop that iterates over each node in the `pos` dictionary, returning pairs of `(node, (x, y))` where `node` is the species name and `(x, y)` are its coordinates in the graph.

`os.path.join(silhouettes_dir, f”{node}.jpg”)` creates a path to the silhouette image file. The script assumes that the silhouette image files are named according to their species, such as ‘fox.jpg’. The image extension `.jpg` can be altered according to the desired file type.

The `get_image()` function is then used to load and process the image and ensures that all images have the same height (in this instance the `target_height` parameter has been set to `120`).

The embedded ‘if’ block checks if the `get_image()` function has successfully loaded and preprocessed the image, and then places the `OffsetImage` at the `(x, y)` coordinates.

The ‘else’ block ensures that if a silhouette image was not successfully loaded a box containing the species name is placed at the `(x, y)` coordinates instead.

In [ ]:
# Add silhouette images as nodes, or species name if silhouettes are not available
for node, (x, y) in pos.items():
    img_path = os.path.join(silhouettes_dir, f"{node}.jpg")  # Adjust extension as needed (.png, .jpeg)
    image = get_image(img_path, target_height=120)
    if image:
        ab = AnnotationBbox(image, (x, y), frameon=False)
        ax.add_artist(ab)
    else:
        # If no silhouette found, draw the species name with an oval, white background
        bbox_props = dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=2)
        ax.text(x, y, node, ha="center", va="center", rotation=0, size=12, bbox=bbox_props)

## Module: Determining Line Type

The `get_line_style()` function determines the line styles that will be used to represent the volume of close encounters between each species at each location. The edges will be solid for more than ten close encounters, dashed for 5-10 close encounters, and dotted for 1-4 close encounters.

In [ ]:
# Function to determine line style based on the number of encounters
def get_line_style(num_encounters):
    if (num_encounters > 10):  # Define thresholds as per your dataset
        return 'solid'
    elif (5 <= num_encounters <= 10):
        return (0, (5, 5))  # dashed
    else:
        return (0, (1, 3))  # dotted

This code block defines the `draw_curved_edges()` function which takes on the following inputs: `G` (the graph object containing the nodes and edges), `pos` (the dictionary containing node coordinates), `ax` (the matplotlib axis object indicating where the graph will be drawn), and `edge_counters` (a dictionary to keep track of the number of edges between each pair of nodes).

A base curvature radius is set to `0.01` (the lower the value, the less severe the curve of the edges). This value is then manipulated based on the number of edges between each pair of nodes, ensuring that no edge is obscured from view. `line_style = get_line_style(num_encounters)` determines the line type to be used for the edge based on the volume of close encounters between the species pair at the given location.

In [ ]:
# Function to draw curved edges towards the center of the circle with reduced curvature
def draw_curved_edges(G, pos, ax, edge_counters):
    base_rad = 0.01  # Further reduce curvature radius for more subtle curves

    for (u, v, d) in G.edges(data=True):
        color = G[u][v]['color']
        num_encounters = d['weight']

        edge = tuple(sorted((u, v)))  # Ensure consistent edge representation for edge_counters lookup
        rad = min(0.1, base_rad * edge_counters[edge])  # Use edge_counters for slight variation in curvature, capped at 0.1

        line_style = get_line_style(num_encounters)  # Determine line style based on encounter volume

        # Compute the control points
        ctrl_x, ctrl_y = (pos[u][0] + pos[v][0]) / 2, (pos[u][1] + pos[v][1]) / 2
        dx, dy = pos[v][0] - pos[u][0], pos[v][1] - pos[u][1]

        # Adjust control points towards the center even less aggressively
        ctrl_x -= rad * dy
        ctrl_y += rad * dx

        verts = [(pos[u][0], pos[u][1]), (ctrl_x, ctrl_y), (pos[v][0], pos[v][1])]
        codes = [Path.MOVETO, Path.CURVE3, Path.CURVE3]

        path = Path(verts, codes)
        patch = PathPatch(path, facecolor='none', edgecolor=color, lw=1, linestyle=line_style)
        ax.add_patch(patch)

This line calls the previously defined `draw_curved_edges(G, pos, ax, edge_counters)` function. The function iterates through each edge in graph `G` and determines the curvature of each edge based on the `edge_counters` value. It then draws the edges using `matplotlib` and applies the required color and linetype to represent locationID and the volume of close encounters.

In [ ]:
# Draw edges
draw_curved_edges(G, pos, ax, edge_counters)

Here the plot limit for the graph is defined using the `plt.xlim()` and `plt.ylim()` functions. `plt.axis(‘off’)` ensures that axis lines, ticks, and labels will not be displayed in the final graph. This will provide a more legible and visually appealing network graph.

In [ ]:
# Set plot limits
plt.xlim(-2.5, 2.5)  # Adjust based on the graph size
plt.ylim(-2.5, 2.5)
plt.axis('off')

## Module: Creating the Legends and Title

This snippet uses a list comprehension to create a list of legend entries, with each entry associating a unique color with a locationID.

In [ ]:
# Create the legend mapping colors to LocationIDs
# Legend elements for colors
legend_elements_colors = [Line2D([0], [0], color=color, lw=4, label=f'Location ID {loc_id}')
                          for loc_id, color in location_color_map.items()]

This creates an additional legend displaying the line types and associated values regarding the volume of close encounters between a species pair.

In [ ]:
# Legend elements for line types
legend_elements_lines = [Line2D([0], [0], color='black', lw=2, linestyle='solid', label='> 10 encounters'),
                         Line2D([0], [0], color='black', lw=2, linestyle=(0, (5, 5)), label='5-10 encounters'),
                         Line2D([0], [0], color='black', lw=2, linestyle=(0, (1, 3)), label='< 5 encounters')]

Next the script creates a combined legend to display the color associated with each locationID, and the linetype associate for each category of volume of close encounters. Here the specific characteristics of the first legend displaying color and locationID is specified, such as title and font size. The `ax.add_artist()` function creates the legend.

In [ ]:
# Create the combined legend and ensure both legends are displayed
first_legend = ax.legend(handles=legend_elements_colors, title="Location ID", title_fontsize='13', fontsize='12', loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.5)
ax.add_artist(first_legend)

Here the characteristics of the second legend is specified and the `ax.add_artist()` function creates the legend.

In [ ]:
# Add a second legend for the line types
second_legend = plt.legend(handles=legend_elements_lines, title="Encounters", title_fontsize='13', fontsize='12', loc='upper left', bbox_to_anchor=(1.05, 0.8), borderaxespad=0.5)
ax.add_artist(second_legend)

The `plt.suptitle()` function is used to create a centered title for the plot.

In [ ]:
# Title, centered above the graph
plt.suptitle("Close Encounters Across Species and Sites", fontsize=20, y=0.92)

## Module: Saving the Close Encounters Graph

The plot is saved to a file path constructed using the `os.path.join()` and `os.path.dirname()` functions. The arguments within the `plt.savefig()` function ensure that the plot is saved as a pdf file with some padding, and the final line within the code snippet ensures that there are no overlapping elements within the graph and all of the elements are neatly aligned.

In [ ]:
# Save the plot as a .pdf file
output_path_pdf = os.path.join(os.path.dirname(results_file_path), "network_graph.pdf")
plt.savefig(output_path_pdf, format='pdf', bbox_inches='tight', pad_inches=0.5)

plt.tight_layout()

This print statement informs the user that the plot has been successfully generated and saved to the output file path.

In [ ]:
# Print the location where the PDF file was saved
print(f"The PDF file was saved at: {output_path_pdf}")